<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/extract/gradioprompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install easyocr gradio pythainlp
!sudo apt-get install ghostscript

In [ ]:
!pip install langchain langchain_huggingface langchain_community

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

resume_directory = '/content/drive/MyDrive/AIEngineer/resume/resume_LLM'
pdf_files = [os.path.join(resume_directory, f) for f in os.listdir(resume_directory) if f.endswith('.pdf')]

# Print the list of PDF files to verify
print(pdf_files)

['/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume BA 2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume BA 3.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume ba 5.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume ba2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume baa.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 6.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 7.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 3.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 4.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 5.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 1.pdf']


In [9]:
resume_prompt_en = """
You are a highly experienced CEO with a reputation for demanding excellence. Your company only hires the best of the best, and you approach each resume with a critical eye, carefully weighing the value of each candidate.
As you review the resume below for a position within the Technical Labs team at Google, consider whether this candidate truly stands out in a competitive field.

**Job Description:**
- Lead key initiatives and projects within the team.
- Collaborate with cross-functional teams (product, engineering, research) to evaluate new technologies and drive business impact.
- Stay up-to-date with the latest technology trends and developments.
- Conduct research to develop innovative strategies and solutions.
- Support product development and idea validation.
- Work with experts and engineers to create proofs of concept.
- Contribute to professional communities through writing, speaking, and knowledge sharing.

**Qualifications:**
- Proven experience in strategy and execution in areas like AI, data science, business analysis, or engineering.
- Hands-on skills in relevant technologies and tools (e.g., Python, SQL, machine learning, data modeling).
- Experience applying technology to solve real-world problems.
- Strong programming or analytical skills (Python, SQL, R, etc.).
- Excellent problem-solving and communication skills.
- Ability to work in a team, prioritize tasks, and manage time effectively.
- Quick learner with a self-starter attitude.
- Knowledge in one or more industrial domains is a plus.

Resume: {resume}

**Evaluation:**

Score: [Your score here]
Strengths: [Identify the strengths of the resume]
Areas for Improvement: [Identify areas where the resume could be improved]
Overall Impression: [Summarize your overall thoughts on the resume]

**Scoring Scale:**

1-2: Very Poor – Significant gaps in skills or experience.
3-4: Poor – Some relevant skills but not meeting expectations.
5-6: Average – Meets basic requirements but lacks standout qualities.
7-8: Good – Strong qualifications but not exceptional.
9-10: Excellent – Outstanding and exceeds expectations.
"""

In [11]:
import os
import re
import numpy as np
import pandas as pd
import easyocr
import subprocess
from pythainlp.phayathaibert.core import NamedEntityTagger
import gradio as gr
from PIL import Image
import requests
from threading import Semaphore
from concurrent.futures import ThreadPoolExecutor, as_completed
from huggingface_hub import InferenceClient


# Function to convert PDF to images using Ghostscript
def convert_pdf_to_images(pdf_file, output_folder="/content/images", dpi=300):
    try:
        # Ensure the output directory exists
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Command to call Ghostscript
        output_format = os.path.join(output_folder, "page_%03d.png")
        gs_command = [
            "gs",
            "-sDEVICE=png16m",   # Output as PNG images
            f"-r{dpi}",           # Set the resolution (DPI)
            "-o", output_format,  # Output format
            pdf_file              # Input PDF file
        ]

        subprocess.run(gs_command, check=True)

        # Collect the generated image file paths
        image_files = sorted([os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith(".png")])
        return image_files

    except subprocess.CalledProcessError as e:
        print(f"Ghostscript error: {e}")
        return []

# Function to extract text from a single image
def extract_text_from_image(image_path):
    reader = easyocr.Reader(['th', 'en'], gpu=True)  # Enable GPU if available for faster processing
    image = Image.open(image_path)
    image_np = np.array(image)
    result = reader.readtext(image_np)
    sorted_data = sorted(result, key=lambda x: x[0][0][1])  # Sort based on y-coordinate
    plain_text = "\n".join([text for _, text, _ in sorted_data])
    return plain_text

# Function to tag and clean the extracted text
def tag_and_clean_text(text):
    tagger = NamedEntityTagger()  # No device argument, as it uses GPU automatically if available
    unwanted_terms = [
        'ที่อยู่', 'โทรศัพท์', 'อีเมล', 'linkedin', ':', ',', '-', '|',
        'ประวัติส่วนตัว', 'เกี่ยวกับฉัน', 'about me', 'ชื่อ', 'สกุล', 'tell', 'โทร', 'โทรงาน',
        'ชื่อเล่น', 'อายุ', 'วันเกิด', 'พุทธ', 'ศาสนา', 'สัญชาติ', 'phone',
        'ช่องทางการติดต่อ', '_', 're sume', 'resume', 'resu me'
    ]
    unwanted_pattern = '|'.join(map(re.escape, unwanted_terms))

    try:
        ner = tagger.get_ner(text, tag=True)
        pattern = r'<(?!ORGANIZATION|PERCENT|TIME)[^>]+>[^<]+</[^>]+>'
        cleaned_ner = re.sub(pattern, '', ner)
        cleaned_ner = re.sub(r'</?(ORGANIZATION|PERCENT|TIME)>', '', cleaned_ner)
        cleaned_ner = re.sub(unwanted_pattern, '', cleaned_ner)
        cleaned_ner = re.sub(r'\bal\b', 'ai', cleaned_ner, flags=re.IGNORECASE)
        return ner.strip(), cleaned_ner.strip()
    except Exception as e:
        print(f"Error in tag_and_clean_text: {e}")
        return "", text

# Function to process a single PDF
def process_single_pdf(pdf_file):
    results = []
    try:
        # Convert PDF to images using Ghostscript
        images = convert_pdf_to_images(pdf_file.name)
        raw_text = ""
        for image_path in images:
            raw_text += extract_text_from_image(image_path) + "\n"

        # Tag and clean text
        tagged_text, cleaned_text = tag_and_clean_text(raw_text)

        # Generate evaluation and score
        evaluation = generate_answer(cleaned_text)
        score = extract_score(evaluation)

        results.append({
            "PDF File": os.path.basename(pdf_file.name),
            "Raw_Text": raw_text,
            "Tagged_Text": tagged_text,
            "Cleaned_Text": cleaned_text,
            "Evaluation": evaluation,
            "Score": score
        })

    except Exception as e:
        print(f"Error in process_single_pdf: {e}")

    # Convert results to a DataFrame
    df = pd.DataFrame(results)
    return df

# Function to process multiple PDFs
def process_multiple_pdfs(pdf_files):
    all_results = pd.DataFrame()
    for pdf_file in pdf_files:
        df = process_single_pdf(pdf_file)
        all_results = pd.concat([all_results, df], ignore_index=True)
    return all_results

# Save DataFrame to CSV
def save_dataframe(df):
    try:
        csv_path = '/content/output.csv'
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        return csv_path
    except Exception as e:
        print(f"Error saving DataFrame: {e}")
        return ""

# Semaphore to limit concurrent API requests
semaphore = Semaphore(50)  # Adjust the number as needed

# Define the model parameters
model_params = {
    "max_new_tokens": 512,
    "temperature": 0.5,
    "top_p": 0.95,
    "repetition_penalty": 1.0
}

def generate_answer(resume):
    formatted_prompt = resume_prompt_en.replace("{resume}", resume)
    client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
    response = client.text_generation(formatted_prompt, **model_params)
    output = "".join(response)
    return output

def extract_score(text):
    match = re.search(r'Score:\s*([0-9]+(?:\.[0-9]+)?)', text)
    if match:
        return float(match.group(1))
    return text

# Batch processing with threading
def generate_prompt_answer_optimized(row):
    resume = row.get('Cleaned_Text', "")
    result = generate_answer(resume)
    return result

# Define the batch processing function
def process_batch(batch_df):
    return [generate_prompt_answer_optimized(row) for _, row in batch_df.iterrows()]

# Batch processing with threading
def batch_process(df, batch_size=32):
    results = [None] * len(df)  # Initialize a list to store results in original order
    with ThreadPoolExecutor() as executor:
        futures = {}
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i + batch_size]
            future = executor.submit(process_batch, batch_df)
            futures[future] = (i, i + batch_size)  # Store the index range for each future

        for future in as_completed(futures):
            start_idx, end_idx = futures[future]
            batch_results = future.result()
            results[start_idx:end_idx] = batch_results

    return results

# Gradio Interface Function
def gradio_interface(pdf_files):
    try:
        df = process_multiple_pdfs(pdf_files)
        df['Evaluation'] = batch_process(df, batch_size=32)
        df['Score'] = df['Evaluation'].apply(extract_score)
        csv_path = save_dataframe(df)
        return df, csv_path
    except Exception as e:
        print(f"Error in gradio_interface: {e}")
        return pd.DataFrame(), ""

# Gradio UI
iface = gr.Interface(
    fn=gradio_interface,
    inputs=[gr.Files(label="Upload PDF Files")],  # Allow multiple files
    outputs=[
        gr.Dataframe(label="Extracted and Cleaned Data", height=500, min_width=800),
        gr.File(label="Download CSV")  # Add a file download button
    ],
    title="PDF to Text and Data Cleaner"
)

# Launch the interface
iface.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://282349faec8390ff9b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://282349faec8390ff9b.gradio.live


> 16 mins

In [12]:
import pandas as pd
check = pd.read_csv('/content/output.csv')
check

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text,Evaluation,Score
0,Resume BA 2.pdf,"พิมพ์นารา วัฒนชัย\nที่อยู่: 789 ถนนรัชดาภิเษก,...",<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,st\nเป้าหมายในการทำงาน\nนำความเชี่ยวชาญด้านการ...,Evaluation:\n\nScore: 6\nStrengths: \n- The ca...,6.0
1,Resume ba 5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\n nattamon. rad...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ\n มุ่งมันในการนำทักษะด้านการจัดก...,Score: 3\n\nStrengths: \nThe candidate has exp...,3.0
2,resume 6.pdf,linkedin:\nนภัสสร วิวัฒนาวงศ์\nlinkedin.com i...,linkedin:<PERSON>\n</PERSON><PERSON>น</PERSON>...,in\n\n วิศวกรปัญญาประดิษฐ์\n ประสบการณ์การทำงา...,Score: 6\n\nStrengths:\n- The candidate has a ...,6.0
3,Resume BA 3.pdf,จิตรลดา\n086-234-5678\njitlada.s@domain. com\...,<PERSON> </PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม\n business analyst\nเป้าหมายในการทำงาน\n...,**Evaluation:**\n\nScore: 5\nStrengths: \n- Th...,5.0
4,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit .m@do...,<PERSON> </PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ\nเป้าหมายในการทำงาน\nflnancial a...,11-12: Exceptional – Demonstrates exceptional ...,3.0
5,Resume baa.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทำงาน\nมุ่งมันที่จะใช้ความรู้และท...,Assistant: **Evaluation:**\n\nScore: 5\nStreng...,5.0
6,resume 2.pdf,วงศ์เจริญ\nณ็ชพล\n วิศวกรปัญญาประดิษฐ์\n ประว...,<PERSON> วงศ์เจริญ\nณ็ชพล</PERSON>\n วิศวกรปัญ...,วิศวกรปัญญาประดิษฐ์\n ประวัติการศึกษา\nมหาวิทย...,11-12: Exceptional – Rarely seen skills and ac...,8.0
7,resume 3.pdf,re sume\nประวัติส่วนตัว\n090-123-4567\n pimcha...,re sume\nประวัติส่วนตัว<PHONE>\n09</PHONE><PHO...,ผู้ เชี่ยวชาญด้าน ai ที่มีประสบการณ์ 4 เขี\nใน...,11-12: Exceptional – Rarely seen level of expe...,6.0
8,resume 4.pdf,อาทิตย์\nรัตนวิจิตร\n วิศวกรปัญญาประดิษฐ์\nปร...,<PERSON> </PERSON><PERSON>อาทิตย์\nรัตนวิจิตร<...,วิศวกรปัญญาประดิษฐ์\nประสบการณํ ทำงาน\nวิศวกร ...,**Evaluation:**\n\nScore: 7\nStrengths: \n- Th...,7.0
9,resume 7.pdf,ปัญญา วิริยะชัย\n วิศวกรปัญญาประดิษฐ์\nประสบกา...,<PERSON>ป</PERSON><PERSON>ปัญญา วิริยะชัย</PER...,วิศวกรปัญญาประดิษฐ์\nประสบการณ์การฝึกงาน\nบัณฑ...,Evaluation:\n\nScore: 6\nStrengths: \n- The ca...,6.0


In [8]:
print(check['Evaluation'].iloc[0])

11-12: Exceptional – Exceptional skills and experience, ideal candidate.

**Note:** The scoring scale is out of 12 points, with each category (Experience, Skills, and Qualifications) being scored out of 4 points.

**Interview Questions:**

1. Can you describe a time when you successfully led a project or initiative that had a significant impact on your organization?
2. How do you stay updated with the latest technology trends and developments in your field?
3. Can you provide an example of how you have applied technology to solve a complex business problem?
4. Describe a situation where you had to work with a cross-functional team to achieve a goal. What challenges did you face, and how did you overcome them?
5. How do you prioritize tasks and manage your time effectively when working on multiple projects simultaneously?
6. Can you share an instance where you had to learn a new skill or tool quickly to complete a project? How did you approach the learning process?
7. Tell us about a ti